**Topic Modelling(Konu modelleme)**


*   TopicModelling, büyük metin koleksiyonlarından otomatik olarak gizli temalar ortaya çıkarmak için kullanılan bir NLP yöntemidir.
*   Denetimsiz bir yöntemdir; yani veri etiketlenmemiştir.

Topic Modelling Kullanım alanları

*   Sosyal medya duygu ve konu analizi
*   Müşteri yorumlarının incelenmesi
*   Arama motoru indexleme
*   Bilimsel yayın analizi
*   Liste öğesi
*   Haber ve makale sınıflandırma


**LDA (Latent DirichletAllocation)**
Bu belgelerde hangi konular vardır ve bu konuları hangi kelimeler temsil etmektedir?

•LDA iş akışı•1. Veri Hazırlama, metin temizleme ve önişleme•2.

1.   Veri Hazırlama metin temizleme ve önişleme
2.   Metnin Sayısal Temsili (Bag-of-Words(BoW), TF-IDF ...)
2.   LDA Modelinin Eğitilmesi •Konu sayısı (k) belirlenir. •Model eğitilir
4.   Sonuçların Yorumlanması  •Her konunun en baskın kelimeleri çıkarılır. •Belgelerin hangi konulara yakın olduğu belirlenir.


In [1]:
pip install gensim

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.9/27.9 MB 23.2 MB/s eta 0:00:00


In [2]:
pip install pyLDavis

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 22.7 MB/s eta 0:00:00


In [3]:
import pandas as pd
import nltk
from nltk.corpus import stopwords
from gensim import corpora
from gensim.models import LdaModel
import pyLDAvis
import pyLDAvis.gensim

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [5]:
df= pd.read_csv('newsgroups.csv')
df.head()

text_column="content"

documents= df[text_column].astype(str).tolist()

In [10]:
import re

stop_words = set(stopwords.words('english'))
def clean_text(text):
  text = re.sub(r'https\S+|www\S+|https\S+', '', text, flags=re.MULTILINE)
  tokens= text.lower().split()
  tokens=[t for t in tokens if t.isalpha() and t not in stop_words]

  return tokens

texts=[clean_text(doc) for doc in documents]



In [11]:
# sözlük ve corpus oluşturma
dictionary =corpora.Dictionary(texts)

corpus=[dictionary.doc2bow(text) for text in texts ]

In [13]:
#LDA modelini eğitme

lda_model= LdaModel(
    corpus=corpus, #eğitilecek belge-keime matrisi (Bow yapısı)
    id2word=dictionary, # kelime ID -> kelime eşleştirmesii içeren sözlük
    num_topics=20, #çıkarılacak konu sayısı
    passes=10, # tüm corpustan kaç kez geçileceği (modeli iyileştirir)
    chunksize=2000, #her iterasyonda işlenecek belge sayısı
    random_state=42 # sonuçların tekrarlanabilir olması için sabit değer
)

In [16]:
#konuları yazdırma
for topic in lda_model.print_topics(num_words=50):
  print(topic)
  print("\n")

(0, '0.010*"government" + 0.009*"key" + 0.008*"would" + 0.007*"public" + 0.007*"use" + 0.007*"law" + 0.006*"encryption" + 0.005*"people" + 0.005*"clipper" + 0.005*"new" + 0.005*"state" + 0.005*"security" + 0.004*"chip" + 0.004*"right" + 0.004*"used" + 0.004*"gun" + 0.004*"president" + 0.004*"federal" + 0.003*"clinton" + 0.003*"two" + 0.003*"may" + 0.003*"rights" + 0.003*"one" + 0.003*"could" + 0.003*"private" + 0.003*"system" + 0.003*"keys" + 0.003*"legal" + 0.003*"crime" + 0.003*"states" + 0.003*"national" + 0.003*"access" + 0.003*"protect" + 0.003*"number" + 0.003*"us" + 0.003*"court" + 0.003*"enforcement" + 0.003*"bill" + 0.003*"house" + 0.003*"make" + 0.003*"also" + 0.003*"secret" + 0.002*"american" + 0.002*"information" + 0.002*"laws" + 0.002*"even" + 0.002*"white" + 0.002*"keep" + 0.002*"using" + 0.002*"algorithm"')


(1, '0.012*"information" + 0.010*"university" + 0.009*"research" + 0.008*"data" + 0.008*"national" + 0.007*"science" + 0.007*"available" + 0.007*"program" + 0.005*"

In [17]:
#pyLDAvis ile görselleştirme
vis=pyLDAvis.gensim.prepare(lda_model, corpus, dictionary)
pyLDAvis.display(vis)